In [1]:
import import_ipynb
from sklearn.preprocessing import Imputer, MinMaxScaler, OneHotEncoder, LabelEncoder
import numpy as np
import itertools
import pandas as pdraw
import collections
from scipy import sparse
import time
from scipy.sparse import csr_matrix, save_npz, load_npz

from loadData import raw_df, test_df
from Util import getAllTypesofCategory, getAllTypesOfProperty

importing Jupyter notebook from loadData.ipynb
importing Jupyter notebook from Util.ipynb


## impute missing value with most_frequent strategy

In [2]:
contain_missing_value_columns = ["item_brand_id","item_city_id","item_sales_level","user_gender_id","user_age_level",
                                "user_occupation_id","user_star_level","shop_review_positive_rate",
                                "shop_score_service","shop_score_delivery","shop_score_description"]
other_missing_value_columns = ["predict_category_property"]

In [3]:
data_imputer = Imputer(missing_values=-1 , strategy='most_frequent', axis=0)
raw_df[contain_missing_value_columns] = data_imputer.fit_transform(raw_df[contain_missing_value_columns])

## processing  predict_category_property

In [4]:
def generate_predict_category_property_feature(df):
    def process_func(item):
        result = {}
        for i in item.split(';'):
            if ':' in i:
                items = i.split(':')
                category = int(items[0])
                properties = items[1]
                if ',' in properties:
                    result[category] = map(int,items[1].split(','))
                else:
                    result[category] = [int(items[1])]
        return result
    def process_category(item):
        return list(process_func(item).keys())
    def process_property(item):
        _ = process_func(item).values()
        return set(itertools.chain(*_))
    df['predict_categories'] = df['predict_category_property'].apply(lambda item: process_category(item))
    df['predict_properties'] = df['predict_category_property'].apply(lambda item: process_property(item))
generate_predict_category_property_feature(raw_df)

In [5]:
# # impute empty missing value in index 160943  205272  214168 314685
# raw_df['predict_categories'][160943]= ['7908382889764677758']
# raw_df['predict_categories'][205272] = ['7908382889764677758']
# raw_df['predict_categories'][214168] = ['7908382889764677758']
# raw_df['predict_categories'][314685] = ['7908382889764677758']

In [6]:
tmp = raw_df['predict_categories'].values
all_categories = list(set(itertools.chain(*tmp)))
category_label_encode = LabelEncoder()
all_categories_labels = category_label_encode.fit_transform(all_categories)

In [7]:
# raw_df['predict_categories'] = raw_df['predict_categories'].apply(lambda item: category_label_encode.transform(item))

In [8]:
category_label_oh_enc = OneHotEncoder()
category_label_oh_enc.fit(all_categories_labels.reshape(-1,1))

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [9]:
# predict_categories_df = raw_df['predict_categories'].apply(lambda item: np.sum(category_label_oh_enc.transform(item.reshape(-1,1)),axis=0))

In [10]:
# save_npz('../processed_data/predict_categories.npz',  csr_matrix(np.concatenate(predict_categories_df.values).reshape(len(predict_categories_df),-1)))

## processing the object column

In [11]:
# # split category_list to multi features
# def extract_item_category_list_func(item, num):
#     items = item.split(';')
#     if num >=2:
#         if len(items) > 2:
#             return items[num]
#         else:
#             return 0
#     else:
#         return items[num]
# def generate_item_category_features():
#     raw_df['item_one_level_category'] = raw_df['item_category_list'].apply(lambda item: extract_item_category_list_func(item, 0))
#     raw_df['item_second_level_category'] = raw_df['item_category_list'].apply(lambda item: extract_item_category_list_func(item, 1))
#     raw_df['item_third_level_category'] = raw_df['item_category_list'].apply(lambda item: extract_item_category_list_func(item, 2))
# #     raw_df.drop('item_category_list', axis=1)
# generate_item_category_features()

In [12]:
# raw_df[['item_one_level_category', 'item_second_level_category', 'item_third_level_category']].to_csv('../processed_data/three_level_category.csv', sep=' ',index=False)

In [13]:
# extract_item_property_list_func = lambda item: list(item.split(';'))
# raw_df['item_property_list_array'] = raw_df['item_property_list'].apply(lambda item: extract_item_property_list_func(item))

In [14]:
# np.savetxt('../processed_data/item_property_list_array.txt', raw_df['item_property_list_array'].reshape(len(raw_df['item_property_list_array']), -1), fmt='%s')

## generate new raw_df called _raw_df after filling missing value and handle category_id and property _id, drop useless column

In [15]:
# raw_df = raw_df.drop(['item_category_list','item_property_list'], axis=1)

In [16]:
# need_onehot_columns = ['item_brand_id','item_city_id','user_gender_id','item_one_level_category', 'item_second_level_category']
# special_onehot_columns = ['item_property_list_array', 'item_third_level_category']
# other_columns = ['context_timestamp','predict_category_property','is_trade']
# need_scale_columns = ['item_price_level','item_sales_level','item_collected_level','item_pv_level',
#                      'user_age_level','user_occupation_id','user_star_level','context_page_id',
#                      'shop_review_num_level','shop_review_positive_rate','shop_star_level','shop_score_service',
#                     'shop_score_description', 'day', 'hour']

In [17]:
# #transform to label type
# for i in ['item_brand_id','item_city_id','user_gender_id']:
#     raw_df[i] = raw_df[i].apply(lambda item : str(item)) 

## One hot encoding

In [20]:
# need_onehot_df = pd.get_dummies(raw_df[need_onehot_columns], prefix=need_onehot_columns)
# save_npz('../processed_data/need_onehot_columns.npz', csr_matrix(need_onehot_df.values))

#### item_third_level_category process

In [21]:
# third_level_items = raw_df['item_third_level_category'].unique()

# lb = LabelEncoder()
# _tmp = lb.fit_transform(third_level_items)
# result = lb.transform(raw_df['item_third_level_category'].values)

# ohenc = OneHotEncoder()
# ohenc.fit(_tmp.reshape(-1,1))
# result = ohenc.transform(result.reshape(-1,1))

# tmp_df = pd.DataFrame(data=result.toarray(), columns=['item_third_level_None', 'item_third_level_8868887661186419229', 'item_third_level_6233669177166538628'])

# tmp_df.to_csv('../processed_data/item_third_level_one_hot.csv', sep=' ',float_format='%d',index=False)

####  handle property list , label encode first and then onehot encoder

In [22]:
# arrays = list(raw_df['item_property_list_array'])
# count = collections.Counter(list(itertools.chain(*arrays)))
# most_common_property_id_list = list(list(zip(*count.most_common(1499)))[0])
# most_common_property_id_list.append('o')

In [23]:
# le = LabelEncoder()
# le.fit(most_common_property_id_list)
# most_common_property_id_num_list = le.transform(most_common_property_id_list)

In [24]:
# # label property id in item_property_list_array
# for i in range(raw_df.shape[0]):
#     for index, _id in enumerate(raw_df['item_property_list_array'][i]):
#         if _id not in most_common_property_id_list:
#             raw_df['item_property_list_array'][i][index] = 'o'

In [25]:
# raw_df['item_property_list_array'] = raw_df['item_property_list_array'].apply(lambda item:le.transform(item))

In [26]:
# enc = OneHotEncoder()
# enc.fit(most_common_property_id_num_list.reshape(-1,1))

In [27]:
# _tmp_df = raw_df['item_property_list_array'].apply(lambda item: np.sum(enc.transform(item.reshape(-1,1)),axis=0).reshape(1,-1)[0])
# save_npz('../processed_data/item_property.npz', csr_matrix(np.concatenate(_tmp_df.values).reshape(len(_tmp_df),-1)))

## scaling data in need_scale_column

In [28]:
# # min-max normalization
# _tmpdf = raw_df[need_scale_columns]
# # normalized_df = (_tmpdf - _tmpdf.min())/(_tmpdf.max() - _tmpdf.min())

# # mean normalization
# normalized_df = (_tmpdf - _tmpdf.mean())/_tmpdf.std()

In [29]:
# normalized_df.to_csv('../processed_data/scaled.csv', sep=' ',index=False)